In [224]:
from urllib.request import urlopen
from urllib.error import HTTPError
import re
from datetime import datetime

In [225]:
mensas = {
    "Untere Mensa": "zentrum-mercato",
    "Obere Mensa": "zentrum-mensa",
    "Platte 14": "platte-14",
    "Rämi 59": "raemi59",
    "Zahnmedizin": "cafeteria-zzm",
    } # Polymensa, Clausius, Foodtruck

In [226]:
def wochentag():
    wochentage = ["montag", "dienstag", "mittwoch", "donnerstag", "freitag"]
    tag_num = datetime.today().weekday()

    if tag_num >= 5:
        print("Mensas are closed over the weekend. (I think so)")
        raise AssertionError
    
    return wochentage[tag_num]

In [227]:
# !!! check if correct day
def get_html(mensa_id):
    """
    returns html of menu for mensa with id mensa_id
    """
    url = "https://www.mensa.uzh.ch/de/menueplaene/" + mensa_id + "/" + wochentag()
    
    try:
        page = urlopen(url)
    except:
        # !!! make error more specific
        print("Something didn't work with this webpage (id={}). Most likely the id is incorrect.\nPlease correct or remove this mensa and try again.".format(mensa_id))
        raise

    html = page.read().decode("utf-8")

    return html

In [228]:
def get_menu_name(source):
    """
    gets menu names from source, formats them and returns them as list of strings
    """
    
    pattern = "<h3>.*?<span>"
    menu_name = re.findall(pattern, source, re.IGNORECASE)
    for i in range(len(menu_name)):
        menu_name[i] = re.sub("<.*?>", "", menu_name[i])
    
    return menu_name

In [229]:
def get_menu(source):
    """
    gets menues from source, formats them and returns them as list of strings
    """

    pattern = "</span> </h3>.*?</p>"
    menu = re.findall(pattern, source, re.DOTALL)

    for i in range(len(menu)):
        menu[i] = re.sub("Fleisch:.*?$", "", menu[i])
        menu[i] = re.sub("<br>", ",", menu[i])
        menu[i] = re.sub("<.*?>", "", menu[i])
        menu[i] = re.sub("\n", "", menu[i])
        menu[i] = re.sub(" ,", ",", menu[i])
        menu[i] = re.sub("^ *", "", menu[i])
        menu[i] = re.sub(", *$", "", menu[i])

    return menu

In [230]:
def menu_uzh(mensas):
    """
    Prints menues of all mensas in "mensas"
    """
    for mensa, id in mensas.items():
        html = get_html(id)
        menu_name = get_menu_name(html)
        menu = get_menu(html)
        
        print("|||  ", mensa, "  |||\n")
        for i in range(len(menu_name)):
            print("   " + menu_name[i] + ":\n\n        " + menu[i], "\n\n")
    return


In [231]:
menu_uzh(mensas)

|||   Untere Mensa   |||

   garden:

        Plantet Tikka Masala, Basmati Reis, Kokosnuss Flocken, Menüsalat oder Apfelmus 


   pure asia:

        Yakisoba mit Pouletgeschnetzeltes, Wok Gemüse und Sauce, Menüsalat oder Apfelmus 


   butcher:

        Poulet Hackbraten, Portweinsauce, Kartoffelstock, Erbsen-Lauch 


|||   Obere Mensa   |||

   farm:

        Tofu Gyros, Joghurtsauce, Bulgursalat, Weiss Kabissalat 


   butcher:

        Poulet Hackbraten, Portweinsauce, Kartoffelstock, Erbsen-Lauch 


|||   Platte 14   |||

   Häppy vegan bowl:

        Linseneintopf, Kartoffeln, Lauch, Kurkumasauce, Bürli 


   Lucky bowl:

        Ofen Poulet-Schenkel, Wassermelonen Feta Salat, Sesam Joghurt, Poulet: Schweiz 


|||   Rämi 59   |||

   Traditional:

        Geschmorte Randen, Lyoner Kartoffeln, Joghurtsauce 


   Global:

        Gemüse Tajine, mit Kichererbsen, Bulgur, Koriander 


|||   Zahnmedizin   |||

   farm:

        Kartoffeltortilla, Kräuterquark, Gartenkresse, Menusalat